In [21]:
## Each Space is a cell in a juptyr notebook ("QuickThinking" in fastai_workbench) that runs on Gradient Paperspace
from greek_normalisation.utils import (nfd, nfc, nfkc, convert_to_2019)
from greek_accentuation.characters import base
from cltk.lemmatize.grc import GreekBackoffLemmatizer
from alphabet import filter_non_greek, filter_non_greek_with_punctuation
from alphabet import PUNCTUATION
from cltk.stops.grc import STOPS
from greek_swadesh import SWADESH
from chrysostom_junk_words import JUNK_WORDS
import numpy as np

#greek_text = 'ὁ αὐτὸς μῆνιν ἄειδε θεὰ Πηληϊάδεω Ἀχιλῆος; \nκαὶ κατέβην χθὲς εἰς Πειραιᾶ μετὰ Γλαύκωνος τοῦ Ἀρίστωνος. Τιμταμ;'
file = open('/notebooks/storage/greek_text/ALL_REAL_JOHN_FOR_BAG_OF_WORDS.TXT')
greek_text = file.read().lower()
file.close()
all_text = greek_text.split('.')
print("total sentences:")
total_sentences = len(all_text)
print(total_sentences)
no_of_sentences = total_sentences * 0.25
no_of_sentences = total_sentences * 0.50
#no_of_sentences = total_sentences * 0.75
selection = all_text[0:int(no_of_sentences)] #grab only the first XXXX sentences for testing
print("selected sentences:")
print(len(selection))
percent_selected = (len(selection) / len(all_text) * 100)
print("percent selected for processeing:")
print(percent_selected)
greek_text = '.'.join(selection)
original_text = greek_text
lemmatizer = GreekBackoffLemmatizer()

def strip_diacriticals(greek_text):
    return ''.join([base(char) for char in greek_text])

def normalize_greek(greek_text):
    greek_text = nfc(greek_text) #normalize pre-composed unicode characters to be single characters
    greek_text = convert_to_2019(greek_text).lower() #normalize apostrophes and lowercase
    greek_text = filter_non_greek_with_punctuation(greek_text) #strip everything but greek and basic punctuation
    greek_text = greek_text.replace(';','.').replace('·','.') #replace all punctuation except ","s with "."s
    return greek_text

def unhelpful_words():
    stops = sorted(normalize_greek(' '.join(STOPS)).split(' '))
    swadesh = sorted(normalize_greek(' '.join(SWADESH)).split(' '))
    empty = ['', ' ', '\n']
    common = ['πρό','δήποτε', 'δεύτερος', 'αἰών','τοσοῦτος', 'ποτέ','ἴσως', 'ἐκεῖνος','ἐπεί','ἀνά','εἴτε','φησὶ,','ἰδοὺ', 'εἶπε,', 'πῶς,', 'τοσοῦτον,', 'ὅσος','μηδείς', 'μηδέ', 'μήν','γὰρ,', 'εἶπον','ἐστὶν,','ἐστὶν', 'ὅτε','ποῖος','μοι,','ἐρῶ','τοῦτο,','ἀντί','πόσος','ἐστιν','εἶπεν','εὐθύς','ποῦ','ἔμπροσθεν','ἀπαιτέω','κἀν','διό' 'ποτέ' 'ἵημι' 'ἡμῶν,' 'σαφής' 'ὅπου', 'γίγνομαι', 'ἵημι', 'ἐντεῦθεν','ἕνεκα','σου','δοκέω','φησὶ', 'φημί','τίθημι','γὰρ','τοσοῦτος','ὅσπερ','πᾶς','μόνος','ὅταν','ἅπας','δύναμαι','καθά','τοσοῦτος','ἀρετή','ἡμέτερος','ἀεί','τουτέστι,','πλῆθος','αὐτὸν', 'εἶναι', 'ἦν', 'ἐστιν,' 'ἐπειδὴ', 'αὐτῷ', 'αὐτοὺς', 'τότε', 'τούτου', 'πάντες', 'αὕτη', 'ἦν', 'πάντας', 'ὅμως', 'φησιν', 'τουτέστι', 'ἔχειν', 'πλέον', 'ἑαυτοὺς', 'οὕτω', 'οὐχ', 'ὥσπερ', 'ἐστι', 'ἐστιν', 'μόνον', 'τοῦτό', 'ἔχει', 'ὑμᾶς', 'ἐστὶ', 'μέγα', 'γῆς', 'ποιεῖν', 'τοσοῦτον', 'ὅσον', 'μικρὸν', 'γοῦν', 'γέγονε', 'ἵνα', 'ἡμᾶς', 'ἡμῶν', 'σου', 'σε', 'εἶπεν', 'με', 'πᾶσαν', 'κατ', 'ἐκεῖνο', 'εἶπεν', 'αὐτῇ', 'γένοιτο', 'καθάπερ', 'ἔστι', 'ἐστιν', 'ὑμῶν', 'ἕτερον', 'οὗ', 'ἀεὶ',  'οἷς', 'λέγοντος', 'ἐμοὶ', 'νῦν', 'ταῦτα', 'πάντα', 'μόνον', 'ἡμῖν', 'τούτων', 'πάντων', 'οὐχὶ', 'ἐκείνων', 'μηδὲν', 'δὲ', 'τοῦτον', 'ἐστὶν',  'ἐκεῖνα', 'ἆρα', 'μετ', 'λέγων', 'α', 'ἄλλως', 'οὐδὲν', 'φησὶ', 'πάλιν', 'αὐτοῖς', 'εἶτα', 'πολλὰ', 'καλῶς', 'ἃ', 'τοιοῦτον', 'ἐκεῖνοι', 'εἶναι', 'χρὴ', 'μείζονα', 'τουτέστιν', 'αὑτοῦ', 'πάντοθεν', 'αἰτίαν', 'τοῦτο', 'τὰς', 'αἱ', 'μηδὲ', 'αὐτὴν', 'καίτοι', 'ἄλλων', 'ποιεῖ', 'γενέσθαι', 'ποτε', 'εἶπε', 'παντὸς', 'ἐκείνην', 'οἷον', 'ἀλλὰ', 'ἕκαστος', 'εἰπεῖν', 'φησὶν', 'δι', 'τοίνυν', 'ταύτην', 'πάλιν', 'ὧν', 'σφόδρα', 'σοι', 'εὐθέως', 'δῆλον', 'πολλὴν', 'ἕως', 'πόθεν', 'ἤδη', 'οὖν', 'εἰσιν', 'τὸ', 'οὐδέν', 'δεῖ', 'γὰρ', 'ὃ', 'λέγων', 'μου', 'τούτῳ', 'ἐπεὶ', 'πρῶτον', 'λέγει', 'οὐκοῦν', 'αὐτοὶ', 'οὗτοι', 'γ', 'αὐτόν', 'φησι', 'ταῖς', 'ὅπερ', 'ὁρᾷς', 'αὐτῆς', 'ὄντως', 'ἐκείνου', 'μήτε', 'μοι', 'διὸ', 'χωρὶς', 'ἅμα', 'εἰπέ', 'εἰπέ_μοι', 'ἔσται', 'ὁμοίως', 'ἐκείνῳ',  'αὐτοῦ', 'αὐτῶν', 'παρ', 'γίνεται', 'λέγω', 'οὐκέτι', 'τοιαῦτα', 'εἰκότως', 'αὐτοῦ', 'ἑτέρων', 'β', 'ἐκείνης', 'μὴν', 'πολλῆς', 'πολλάκις', 'ἁπλῶς', 'ἕνεκεν', 'αὐτὰ', 'ὑμῖν', 'τίνος', 'πάσης', 'φησι', 'μυρία', 'τίνος_ἕνεκεν', 'ἔχων', 'ἧς', 'νῦν', 'αὐτὸ', 'ἔστιν', 'φησίν', 'καθ', 'λέγει', 'μέχρι', 'πρὸ', 'πᾶν', 'τοῦτο', 'ἐκεῖνον', 'ἅπερ', 'πολλοὶ', 'ἄλλο', 'ἦσαν', 'ἔλεγεν', 'ἀμήν', 'μοι', 'μάλιστα', 'ταύτης', 'ἐστι', 'πανταχοῦ', 'φησί', 'λοιπὸν', 'εἰπεῖν', 'τινες', 'πολὺ', 'ἐστίν', 'ἅπαντα', 'ἀντὶ', 'οὐδαμοῦ', 'ἀλλήλους', 'μᾶλλον', 'κἂν', 'πολλῷ', 'εἴ', 'πολλῷ', 'μᾶλλον', 'νόμον', 'ἔνθα', 'ᾖ', 'τούτοις', 'ταχέως', 'εἶδες', 'ὅλως', 'οὐδέποτε']
    return sorted(list(set(np.concatenate((stops, swadesh, empty, common)))))

UNHELPFUL = unhelpful_words()
#print(UNHELPFUL)

def remove_words_unhelpful_for_lda(lemmata):
    helpful_words = [word for word in lemmata if word not in UNHELPFUL and word != '' and word != None]
    return helpful_words

def tokenize(sentence):
    sentence = sentence.replace(',','').split(' ')
    lemmatized_tuples = lemmatizer.lemmatize(sentence)
    lemmata = [tuple[1] for tuple in lemmatized_tuples]
    lemmata = remove_words_unhelpful_for_lda(lemmata)
    return lemmata

def make_lda_documents(greek_text):
    # LDA wants a List of Lists of words (= document), I will make each sentence a document
    print("chars in greek text:")
    print(len(greek_text))
    print("words in greek text:")
    print(len(greek_text.split(' ')))

    print("normalizing greek...")
    import datetime
    start_normalizing = datetime.datetime.now()
    print(start_normalizing)
    documents = normalize_greek(greek_text).split('.')
    print("...done.")
    done_normalizing = datetime.datetime.now()
    print(done_normalizing)

    print("tokenizing...")
    start_tokenizing = datetime.datetime.now()
    print(start_tokenizing)

    lda_documents = []
    sentences_per_document = 50 # doc ~ a paragraph / thought
    current_document = []

    for sentence in documents:
        if len(sentence) > 0:
            sentence = tokenize(sentence)
        if len(sentence) > 0:
            current_document.extend(sentence)
            if len(current_document) >= sentences_per_document:
                lda_documents.append(current_document)
                current_document = []

    print("...done.")
    done_tokenizing = datetime.datetime.now()
    print(done_tokenizing)

    print("number of lsa_documents:")
    print(len(lda_documents))

    print("peek at lsa_documents:")
    print(lda_documents[-3])


    return lda_documents

total sentences:
141400
selected sentences:
35350
percent selected for processeing:
25.0


In [22]:
#Run Preprocessing
lda_documents = make_lda_documents(greek_text)

chars in greek text:
5702498
words in greek text:
901268
normalizing greek...
2020-10-13 04:40:05.976366
...done.
2020-10-13 04:40:12.886135
tokenizing...
2020-10-13 04:40:12.886640
...done.
2020-10-13 04:40:20.288928
number of lsa_documents:
6057
peek at lsa_documents:
['ἁπλός', 'ταπεινόω', 'συντρίβω', 'συντρίβω', 'διακλάω', 'βούλομαι', 'ἐπαίρω', 'ταπεινόω', 'ψυχή', 'συντρίβω', 'κατανύσσομαι', 'συντρίβω', 'οἰκεῖος', 'ἁμάρτημα', 'μιμνήσκω', 'συνεχής', 'ταπεινόω', 'βούλομαι', 'τύφω', 'ἀνίστημι', 'χαλινόω', 'σύνοιδα', 'ἐπιλαμβάνω', 'διάἐγείρω', 'καταστέλλω', 'πείθω', 'μετριάζω', 'θεός', 'χάρις', 'εὑρίσκω', 'ταπεινόω', 'σαυτοῦ', 'ἐναντίον', 'κύριος', 'εὑρίσκω', 'χάρις', 'θεός', 'χάρις', 'εὑρὼν', 'αἰσθάνομαι', 'χαλεπός', 'θεός', 'χάρις', 'ἐκεῖνος', 'εὐκολία', 'διατρέχω', 'δεινός', 'κεῖμαι', 'ἁμαρτάνω', 'διαφεύγω', 'τιμωρία', 'χάρις', 'θεός', 'προηγέομαι', 'ἐξευμενίζω', 'ἀπολαύω', 'χριστός', 'ἰησοῦς', 'κύριος', 'δόξα', 'ἅγιος', 'πνεῦμα']


In [23]:
#Compute bigrams
from gensim.models import Phrases
# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(lda_documents, min_count=20)
for idx in range(len(lda_documents)):
    for token in bigram[lda_documents[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            lda_documents[idx].append(token)

2020-10-13 04:40:22,653 : INFO : collecting all words and their counts
2020-10-13 04:40:22,654 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2020-10-13 04:40:23,196 : INFO : collected 264129 word types from a corpus of 341176 words (unigram + bigrams) and 6057 sentences
2020-10-13 04:40:23,197 : INFO : using 264129 counts as vocab in Phrases<0 vocab, min_count=20, threshold=10.0, max_vocab_size=40000000>


In [24]:
#Remove rare and common tokens (Filter out words that occur less than 20 documents, or more than 50% of the documents)
from gensim.corpora import Dictionary
lda_dictionary = Dictionary(lda_documents)
lda_dictionary.filter_extremes(no_below=20, no_above=0.5)

2020-10-13 04:40:27,326 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-10-13 04:40:27,685 : INFO : built Dictionary(17713 unique tokens: ['αγιοισ', 'αρχιεπισκοπου', 'βαπτίζω', 'γράμμα', 'δευτεραν']...) from 6057 documents (total 345354 corpus positions)
2020-10-13 04:40:27,980 : INFO : discarding 15147 tokens: [('αγιοισ', 13), ('αρχιεπισκοπου', 12), ('δευτεραν', 2), ('εισ', 18), ('εν', 13), ('επιστολην', 10), ('ημων', 13), ('θεός', 3042), ('ιωαννου', 13), ('κῦμα', 4)]...
2020-10-13 04:40:27,980 : INFO : keeping 2566 tokens which were in no less than 20 and no more than 3028 (=50.0%) documents
2020-10-13 04:40:27,987 : INFO : resulting dictionary: Dictionary(2566 unique tokens: ['βαπτίζω', 'γράμμα', 'εἰρηνέω', 'ζωή', 'θέλημα']...)


In [25]:
#Vectorize documents into Bag-of-words.
lda_corpus = [lda_dictionary.doc2bow(doc) for doc in lda_documents]
#Let’s see how many tokens and documents we have to train on.
print('Number of unique tokens: %d' % len(lda_dictionary))
print('Number of documents: %d' % len(lda_corpus))

Number of unique tokens: 2566
Number of documents: 6057


In [26]:
#5. Train LDA model.
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html#sphx-glr-auto-examples-tutorials-run-lda-py
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim.models import LdaModel

# Set training parameters.
num_topics = 50 #15
chunksize = 1500
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = lda_dictionary[0]  # This is only to "load" the lda_dictionary.
id2word = lda_dictionary.id2token

model = LdaModel(
    corpus=lda_corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

top_topics = model.top_topics(lda_corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)
from pprint import pprint
pprint(top_topics)

2020-10-13 04:40:39,015 : INFO : using autotuned alpha, starting with [0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]
2020-10-13 04:40:39,018 : INFO : using serial LDA version on this node
2020-10-13 04:40:39,035 : INFO : running online (multi-pass) LDA training, 50 topics, 20 passes over the supplied corpus of 6057 documents, updating model once every 1500 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2020-10-13 04:40:39,036 : INFO : PROGRESS: pass 0, at document #1500/6057
2020-10-13 04:40:41,311 : INFO : optimized alpha [0.018842, 0.018624514, 0.018464973, 0.018638296, 0.018639268, 0.018639162, 0.018884556, 0.018349621, 0.018436635, 0.018610096, 0.018436136, 0.01849384, 0.0

2020-10-13 04:40:44,692 : INFO : topic #40 (0.022): 0.028*"ποιέω" + 0.018*"κύριος" + 0.016*"εἰκών" + 0.012*"λαλέω" + 0.011*"λόγος" + 0.011*"ῥῆμα" + 0.011*"γραφή" + 0.011*"βουλεύω" + 0.011*"προφήτης" + 0.010*"ὁράω"
2020-10-13 04:40:44,692 : INFO : topic #16 (0.022): 0.026*"δεσπότης" + 0.025*"ἐπιδείκνυμι" + 0.025*"λαμβάνω" + 0.022*"παράδεισος" + 0.020*"ἡσαῦ" + 0.018*"ἰακὼβ" + 0.017*"φιλανθρωπία" + 0.016*"ἀβραὰμ" + 0.014*"σκοπάω" + 0.014*"ὑπισχνέομαι"
2020-10-13 04:40:44,693 : INFO : topic diff=0.215096, rho=0.500000
2020-10-13 04:40:44,703 : INFO : PROGRESS: pass 0, at document #6057/6057
2020-10-13 04:40:44,749 : INFO : optimized alpha [0.020598525, 0.020880373, 0.020709133, 0.020212248, 0.021591049, 0.020136857, 0.021940878, 0.018613167, 0.020588705, 0.021197826, 0.019896775, 0.020942897, 0.021660004, 0.020962559, 0.021842178, 0.021069305, 0.02109373, 0.021173682, 0.020777239, 0.02274335, 0.020314677, 0.020925777, 0.02049896, 0.021492073, 0.020339252, 0.020572666, 0.021248063, 0.02255

2020-10-13 04:40:48,154 : INFO : topic #40 (0.024): 0.026*"ποιέω" + 0.021*"οὐσία" + 0.020*"εἰκών" + 0.018*"προφήτης" + 0.018*"κύριος" + 0.014*"λόγος" + 0.011*"λαλέω" + 0.010*"ὁράω" + 0.010*"ῥῆμα" + 0.009*"δείκνυμι"
2020-10-13 04:40:48,155 : INFO : topic #19 (0.025): 0.048*"χριστός" + 0.048*"κύριος" + 0.046*"ἰησοῦς" + 0.033*"δόξα" + 0.027*"ἅγιος" + 0.025*"χάρις" + 0.025*"πνεῦμα" + 0.017*"φιλανθρωπία" + 0.014*"τιμή" + 0.013*"χριστός_ἰησοῦς"
2020-10-13 04:40:48,156 : INFO : topic diff=0.312176, rho=0.406962
2020-10-13 04:40:48,166 : INFO : PROGRESS: pass 1, at document #6000/6057
2020-10-13 04:40:49,347 : INFO : optimized alpha [0.022413144, 0.022910371, 0.022468805, 0.022474963, 0.02420276, 0.022547834, 0.023620259, 0.021378634, 0.023021508, 0.024508692, 0.021651542, 0.022997336, 0.02362618, 0.0232332, 0.025039693, 0.02360475, 0.025571834, 0.023902578, 0.022727838, 0.025403578, 0.0228769, 0.022680055, 0.02346374, 0.024179904, 0.02261578, 0.022521898, 0.023836141, 0.025197625, 0.02260

2020-10-13 04:40:51,655 : INFO : topic #27 (0.029): 0.047*"παῦλος" + 0.033*"οἶδα" + 0.029*"γράφω" + 0.027*"χριστός" + 0.023*"ἀκούω" + 0.022*"ἅγιος" + 0.020*"μανθάνω" + 0.018*"ἀγνοέω" + 0.018*"ἀπόστολος" + 0.016*"λέγω1"
2020-10-13 04:40:51,656 : INFO : topic diff=0.485511, rho=0.376943
2020-10-13 04:40:51,666 : INFO : PROGRESS: pass 2, at document #4500/6057
2020-10-13 04:40:52,725 : INFO : optimized alpha [0.024822915, 0.025998913, 0.025544632, 0.024620814, 0.027083, 0.024496693, 0.02673291, 0.024031637, 0.024919143, 0.028829485, 0.023717321, 0.024892017, 0.026443012, 0.025630567, 0.027175894, 0.026923072, 0.027780699, 0.027031919, 0.024466543, 0.029274944, 0.025860948, 0.025490496, 0.026614865, 0.027088491, 0.025723552, 0.025239173, 0.027073078, 0.029951995, 0.02512496, 0.029168742, 0.025263084, 0.028677696, 0.028653488, 0.028195787, 0.024686541, 0.025817648, 0.026808523, 0.028073234, 0.027548911, 0.026609972, 0.028413914, 0.024252798, 0.027909169, 0.027681012, 0.026429277, 0.0249456

2020-10-13 04:40:54,983 : INFO : topic diff=0.513908, rho=0.352717
2020-10-13 04:40:54,994 : INFO : PROGRESS: pass 3, at document #3000/6057
2020-10-13 04:40:56,152 : INFO : optimized alpha [0.027223602, 0.029537039, 0.028964037, 0.027336799, 0.030579595, 0.027403746, 0.029955234, 0.026696822, 0.027561653, 0.032574404, 0.026344959, 0.028151328, 0.029312583, 0.027754292, 0.030672234, 0.030668966, 0.03187223, 0.03058628, 0.027700912, 0.032744285, 0.029044779, 0.028120233, 0.0304556, 0.030094959, 0.02804212, 0.027510522, 0.029916732, 0.034241658, 0.027542897, 0.03370366, 0.028229106, 0.032593396, 0.032951098, 0.032162827, 0.027407793, 0.028521951, 0.029643217, 0.031748816, 0.031159652, 0.029273748, 0.030908382, 0.026331972, 0.031539917, 0.031409737, 0.029317804, 0.027939765, 0.028794535, 0.031403307, 0.03018649, 0.028902404]
2020-10-13 04:40:56,159 : INFO : merging changes from 1500 documents into a model of 6057 documents
2020-10-13 04:40:56,168 : INFO : topic #41 (0.026): 0.041*"ψυχή" +

...

2020-10-13 04:41:39,614 : INFO : topic #10 (0.068): 0.148*"νόμος" + 0.057*"πρᾶγμα" + 0.040*"γάμος" + 0.037*"ὑβρίζω" + 0.035*"χράω2" + 0.031*"εὖ" + 0.027*"ἀγορά" + 0.026*"φύσις" + 0.026*"ἱστορία" + 0.024*"σοφός"
2020-10-13 04:41:39,615 : INFO : topic #30 (0.127): 0.041*"γῆ" + 0.020*"ἐργάζομαι" + 0.019*"ἐσθίω" + 0.016*"ξύλον" + 0.016*"πληρόω" + 0.015*"ἁμαρτία" + 0.014*"τοσοῦτος" + 0.012*"ποιέω" + 0.012*"οἶδα" + 0.012*"ἄξιος"
2020-10-13 04:41:39,615 : INFO : topic #9 (0.136): 0.030*"ἡμέρα" + 0.029*"κακία" + 0.024*"κολάζω" + 0.024*"τιμωρία" + 0.021*"κόλασις" + 0.020*"γέεννα" + 0.019*"ποιέω" + 0.018*"μέλλω" + 0.015*"ἀπόλλυμι" + 0.015*"καιρός"
2020-10-13 04:41:39,616 : INFO : topic #44 (0.155): 0.031*"γῆ" + 0.029*"ὁράω" + 0.023*"ποιέω" + 0.021*"βούλομαι" + 0.018*"δείκνυμι" + 0.017*"δεσπότης" + 0.014*"ἐπιδείκνυμι" + 0.014*"διδάσκω" + 0.013*"φύσις" + 0.013*"ἡμέρα"
2020-10-13 04:41:39,617 : INFO : topic diff=0.068151, rho=0.213017
2020-10-13 04:41:39,627 : INFO : PROGRESS: pass 17, at document

2020-10-13 04:41:41,560 : INFO : topic #30 (0.130): 0.041*"γῆ" + 0.021*"ἐργάζομαι" + 0.019*"ἐσθίω" + 0.018*"ξύλον" + 0.017*"πληρόω" + 0.016*"ἁμαρτία" + 0.014*"τοσοῦτος" + 0.013*"ποιέω" + 0.013*"ἄξιος" + 0.012*"οἶδα"
2020-10-13 04:41:41,561 : INFO : topic #9 (0.134): 0.034*"ἡμέρα" + 0.029*"κακία" + 0.026*"τιμωρία" + 0.021*"κολάζω" + 0.020*"κόλασις" + 0.019*"ποιέω" + 0.018*"μέλλω" + 0.018*"γέεννα" + 0.016*"καιρός" + 0.015*"ἁμάρτημα"
2020-10-13 04:41:41,562 : INFO : topic #44 (0.164): 0.033*"γῆ" + 0.030*"ὁράω" + 0.024*"ποιέω" + 0.022*"βούλομαι" + 0.019*"δεσπότης" + 0.018*"δείκνυμι" + 0.015*"ἐπιδείκνυμι" + 0.014*"διδάσκω" + 0.013*"φύσις" + 0.013*"ἡμέρα"
2020-10-13 04:41:41,562 : INFO : topic diff=0.079563, rho=0.208342
2020-10-13 04:41:41,574 : INFO : PROGRESS: pass 18, at document #3000/6057
2020-10-13 04:41:42,182 : INFO : optimized alpha [0.078127675, 0.0983281, 0.07820956, 0.09120163, 0.071743615, 0.08659, 0.07893196, 0.08397389, 0.075068936, 0.14021964, 0.07006232, 0.08250063, 0.0857

2020-10-13 04:41:43,485 : INFO : topic #9 (0.136): 0.034*"ἡμέρα" + 0.032*"κακία" + 0.030*"τιμωρία" + 0.020*"κολάζω" + 0.019*"ποιέω" + 0.019*"κόλασις" + 0.018*"μέλλω" + 0.017*"νῶε" + 0.016*"γέεννα" + 0.016*"ὀδυνάω"
2020-10-13 04:41:43,485 : INFO : topic #44 (0.174): 0.036*"γῆ" + 0.030*"ὁράω" + 0.024*"ποιέω" + 0.022*"δεσπότης" + 0.021*"βούλομαι" + 0.017*"δείκνυμι" + 0.016*"ἐπιδείκνυμι" + 0.015*"διδάσκω" + 0.014*"φύσις" + 0.013*"ἡμέρα"
2020-10-13 04:41:43,486 : INFO : topic diff=0.165368, rho=0.208342
2020-10-13 04:41:43,497 : INFO : PROGRESS: pass 19, at document #1500/6057
2020-10-13 04:41:44,088 : INFO : optimized alpha [0.07909155, 0.09802943, 0.08002027, 0.09276495, 0.07265554, 0.08797983, 0.080540694, 0.08365984, 0.07680783, 0.13775443, 0.07092867, 0.083901964, 0.08718168, 0.07751166, 0.097079545, 0.100667134, 0.11617591, 0.0985188, 0.08275333, 0.07274497, 0.10688992, 0.08004165, 0.108032055, 0.08588038, 0.087882735, 0.07892144, 0.0817112, 0.09758627, 0.09447373, 0.12685405, 0.13414

2020-10-13 04:41:46,029 : INFO : topic #44 (0.197): 0.037*"γῆ" + 0.031*"ὁράω" + 0.024*"ποιέω" + 0.022*"δεσπότης" + 0.021*"βούλομαι" + 0.017*"ἐπιδείκνυμι" + 0.017*"δείκνυμι" + 0.015*"διδάσκω" + 0.014*"φύσις" + 0.014*"ἡμέρα"
2020-10-13 04:41:46,030 : INFO : topic diff=0.075594, rho=0.203963
2020-10-13 04:41:46,040 : INFO : PROGRESS: pass 19, at document #6057/6057
2020-10-13 04:41:46,057 : INFO : optimized alpha [0.07998489, 0.099658884, 0.0812561, 0.0932176, 0.07384502, 0.08915916, 0.08203502, 0.08406972, 0.07887373, 0.1396271, 0.07092899, 0.08604304, 0.08902152, 0.07912319, 0.10002286, 0.101847254, 0.118821815, 0.09927381, 0.08489401, 0.073100224, 0.10721561, 0.081540026, 0.11027101, 0.0871957, 0.088369824, 0.08020751, 0.083625846, 0.0979499, 0.096166454, 0.12928773, 0.13611117, 0.123457484, 0.10431003, 0.11010804, 0.07749256, 0.07883574, 0.073763855, 0.09115245, 0.086944655, 0.07729741, 0.09348664, 0.06740438, 0.094267026, 0.103866465, 0.18150418, 0.099396, 0.11115116, 0.08575947, 0.0

Average topic coherence: -4.2427.
[([(0.036001176, 'γῆ'),
   (0.030738076, 'ὁράω'),
   (0.023889234, 'ποιέω'),
   (0.022414306, 'δεσπότης'),
   (0.021458868, 'βούλομαι'),
   (0.017040282, 'δείκνυμι'),
   (0.01659255, 'ἐπιδείκνυμι'),
   (0.014581889, 'διδάσκω'),
   (0.013943828, 'φύσις'),
   (0.01345366, 'ἡμέρα'),
   (0.011463536, 'σκοπάω'),
   (0.011293026, 'μανθάνω'),
   (0.011046472, 'λοιπός'),
   (0.011005806, 'πρό'),
   (0.010857443, 'ἄγω'),
   (0.010563547, 'ἀνθρώπινος'),
   (0.010252323, 'πατριάρχης'),
   (0.010178971, 'οἰκεῖος'),
   (0.010102158, 'ἐρῶ'),
   (0.009861595, 'λαμβάνω')],
  -2.258852081516459),
 ([(0.041558884, 'γῆ'),
   (0.023460316, 'ἐσθίω'),
   (0.02061077, 'ἐργάζομαι'),
   (0.020539165, 'ξύλον'),
   (0.017629689, 'πληρόω'),
   (0.015718339, 'ἁμαρτία'),
   (0.0148981, 'δημιουργέω'),
   (0.014517582, 'τοσοῦτος'),
   (0.013145677, 'ποιέω'),
   (0.012671582, 'οἶδα'),
   (0.012618345, 'ἄξιος'),
   (0.011919151, 'ἀξιόω'),
   (0.011686782, 'δέχομαι'),
   (0.011234717